# 1.6 - 1.7. Mô tả và đánh giá dữ liệu & Tiền xử lí dữ liệu

* Load toàn bộ review vào một dataframe duy nhất.

In [1]:
import modules.utils as Utils
import modules.processor as Processor
import numpy as np
import pandas as pd
import enchant
import random

from sklearn.utils import shuffle

In [2]:
# Lấy tất cả các directory path của các lần ta tiến hành crawl data
dir_paths = Utils.getAllFolderPath("./data/")

dir_paths

['./data/product_reviews/']

In [3]:
# Đọc toàn bộ các review từ các file csv
reviews = Utils.readReviews(dir_paths)

In [4]:
# # Lưu data của cột Thông_tin_Review và Sao_danh_gia thành 1 file csv mới
# input_file = "./data/product_reviews/Raw.csv"
# output_file = "./data/product_reviews/Review_Shopee.csv"
# columns = ["Thong_tin_Review", "Sao_danh_gia"]
# rename_map = {"Thong_tin_Review": "raw_comment", "Sao_danh_gia": "rating"}

# Utils.extract_columns(input_file, output_file, columns, rename_map)

In [5]:
reviews = reviews.rename(columns = {"review": "raw_comment", "predicted_sentiment": "label"})

In [6]:
print("Tập dữ liệu có {} bình luận.".format(reviews.shape[0]))

Tập dữ liệu có 15920 bình luận.


In [7]:
Processor.printAfterProcess(reviews)

Shape: (15920, 2)
label
1    11987
0     2160
2     1773
Name: count, dtype: int64


* Ta sẽ tạo khoảng trắng cho những từ tiếng Việt bị dính liền trong `normalized_comment`

In [8]:
reviews['normalized_comment'] = reviews['raw_comment'].apply(lambda cmt: Processor.fix_spacing(cmt))

* Đưa tất cả các text về cùng một chuẩn duy nhất, ta có thể làm điều này bằng cách sử dụng unicodedata.normalize() từ package chuẩn unicodedata của Python.

In [9]:
import unicodedata
reviews['normalized_comment'] = reviews['normalized_comment'].apply(lambda cmt: Processor.normalizeComment(cmt))

* Ta cũng sẽ chuẩn hóa cho cột `raw_comment` nhưng không `lower()` chúng.

In [10]:
reviews['raw_comment'] = reviews['raw_comment'].apply(lambda cmt: Processor.normalizeComment(cmt, False))

reviews.head()

,raw_comment,label,normalized_comment
0,"Chất liệu: tốt, đẹp vải cũng sịnMàu...",1,"chất liệu : tốt , đẹp vải cũng sịn m..."
1,Áo 2 lớp đường may khá chắc chắn le...,1,áo 2 lớp đường may khá chắc chắn le...
2,Không nói gì nhiều mng cứ xem ảnh và ...,1,không nói gì nhiều mng cứ xem ảnh và ...
3,"Áo khoác đẹp, không có chỉ thừa. \nVả...",2,"áo khoác đẹp , không có chỉ thừa . \nv..."
4,"Ưng nhất là giao hàng siêu nhanh, minh đ...",1,"ưng nhất là giao hàng siêu nhanh , minh ..."


<hr>

* Các comment của các sản phẩm đôi khi sẽ chứa các URL do người bán hàng chèn vào để giúp khách hàng có thể click vào để xem các mặt hàng khác, chúng là các noise sample mà ta cần phải loại bỏ khỏi dataset.
  ![](./images/04.png)

In [11]:
reviews['contain_url'] = reviews['raw_comment'].apply(lambda cmt: Processor.containsURL(cmt))

In [12]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

filtered_reviews = reviews[reviews['contain_url'] == 1][['raw_comment','contain_url']]
filtered_reviews


,raw_comment,contain_url
828,"Đúng với mô tả: ổnChất liệu: đúngMàu sắc: đúngVASCARA X CELEBRATING LOCAL PRIDE 2023\n\nĐồng hành cùng chương trình Celebrating Local Pride Fall/Winter'23, với sự xuất hiện của người mẫu Dahan Phương Oanh trong thiết kế túi mang tinh thần tối giản nhưng vẫn sang trọng từ Vascara, cùng trang phục đến từ các thương hiệu Việt.\nMinh chứng cho hình ảnh tự do, phóng khoáng, Dahan Phương Oanh nổi bật với sự cá tính, vượt ra khỏi những chuẩn mực thông thường, tự do thể hiện bản sắc riêng.\n\nIn Courtesy of Style Republik and BIDU\nTrang phục: Huelleyrose, K&K\n#freedom #etrelibre #Vascara\n___\n📌Xem thêm tại: https://www.vascara.com/sr-clp-fw23\n📌BST Fall/Winter'23: https://vascara.com/fall-winter-2023\n📌Tham khảo thêm sản phẩm mới: https://www.vascara.com/vascara-new-arrival\n📌Mua sắm tại cửa hàng: https://www.vascara.com/shop\n📌Tiktok: @vascara.official",1
918,"{title=Homework 26.9\n- Hoàn thiện hết online Unit test 1 và UNit 3 trước 12h sáng thứ 6 ngày 29 tháng 9\n- Làm practice test 1- practice test 3 từ trang 149 đến trang 158 sách student book\n- Làm teacher handout phần từ vựng ngữ pháp từ Unit 1 đến Unit 3\n- Luyện tập trên link:https://quizizz.com/join?gc=81541408\nMONG CÁC EM DÀNH THỜI GIAN ÔN TẬP ĐỂ ĐẠT KẾT QUẢ CAO @All, description=, href=, thumb=, childnumber=0, action=rtf, params={""styles"":[{""start"":0,""len"":13,""st"":""b""},{""start"":59,""len"":37,""st"":""c_db342e""}],""ver"":0}, type=}",1
1942,"𝐌𝐈𝐍𝐈𝐆𝐀𝐌𝐄 ""𝐓𝐑𝐔𝐘 𝐓𝐈̀𝐌 𝐎𝐔𝐄𝐑𝐒""\n\n🤓 Kết thúc chuỗi ngày rước đèn trung thu, chắc là giờ đây các bạn tân sinh viên OU đã chuẩn bị sẵn sàng cho chuỗi ngày “rước Deadline” sắp tới rồi đúng không nhỉ? Để giúp các bạn giảm bớt căng thẳng cũng như là bỡ ngỡ cho những ngày đầu nhập học, HappyU sẽ mang đến một Minigame vô cùng thú vị mang tên “𝐓𝐑𝐔𝐘 𝐓𝐈̀𝐌 𝐎𝐔𝐄𝐑𝐒”.\n\n🌻 Tiếp nối chuỗi Series Review các cơ sở học tập, HappyU hy vọng rằng Minigame lần này sẽ giúp các bạn tân sinh viên biết thêm nhiều điều hữu ích, đặc điểm nhận dạng “bạn cùng nhà”, cũng như là mang về cho bản thân những ĐRL đầu tiên của năm học mới nhé.\n\n[𝐓𝐇𝐄̂̉ 𝐋𝐄̣̂ 𝐓𝐇𝐀𝐌 𝐆𝐈𝐀] \n👥 Đối tượng: Tất cả sinh viên đang theo học tại trường Đại học Mở Thành phố Hồ Chí Minh. \n🏆 Quyền lợi: Sinh viên tham gia được cộng 05 ĐRL - Điều 3 vào HK1 năm học 2023 - 2024.\n⏰ Thời gian tham gia: Từ 20h00 ngày 14/10/2023 đến 20h00 ngày 21/10/2023.\n\n[𝐂𝐀́𝐂𝐇 𝐓𝐇𝐔̛́𝐂 𝐓𝐇𝐀𝐌 𝐆𝐈𝐀]\nBước 1️⃣: Like Fanpage HappyU - CLB Kỹ Năng Và Giá Trị Sống.\nBước 2️⃣: Tag 3 người bạn cùng tham gia, Like và Share bài viết ở chế độ công khai cùng Hashtag: \n#HappyU\n#TruyTimOUers\nBước 3️⃣: Lưu Template và Tick vào các hình ảnh mà bạn cho đây là dấu hiệu của một sinh viên OU. Đăng tải lên Story Facebook cá nhân ở chế độ công khai.\nBước 4️⃣: Chụp màn hình và điền minh chứng của bạn vào Form được đính kèm bên dưới. \n𝗟𝗶𝗻𝗸 𝗺𝗶𝗻𝗵 𝗰𝗵𝘂̛́𝗻𝗴: https://forms.gle/ukCiCXb1b7TpHoVz5\n𝗟𝗶𝗻𝗸 𝗺𝗶𝗻𝗵 𝗰𝗵𝘂̛́𝗻𝗴: https://forms.gle/ukCiCXb1b7TpHoVz5\n𝗟𝗶𝗻𝗸 𝗺𝗶𝗻𝗵 𝗰𝗵𝘂̛́𝗻𝗴: https://forms.gle/ukCiCXb1b7TpHoVz5\n\n-------------------------------------------------\nMọi thắc mắc vui lòng liên hệ: \n🌐 Fanpage: HappyU - CLB Kỹ Năng Và Giá Trị Sống \n💌 Email: happyu@ou.edu.vn \n☎️ Hotline: \nNguyễn Tuấn Nam: 0837154865\nPhan Thị Thuỳ Trang: 0358967741\nĐinh Hoàng Cát Tiên: 0768751188\n \n#HappyU\n#TruyTimOUers",1
1943,"MÈN UI ... MẤY BÀ NÉC THÂM, LỖ CHÂN LÔNG TO THÌ BƠI HẾT VÀO ĐÂY NÈ 🤭🤭\nThử theo chiếc trend của các chị đẹp bên Trung dùng dầu em bé để tr.i thâm néc nè 🌸 Hiệu quả được review là rất okkk nhaa

* Bây giờ chúng ta chỉ sẽ lấy các comment mà không chứa URL

In [13]:
reviews = reviews[reviews['contain_url'] == 0]
reviews = reviews.drop(columns=['contain_url']).reset_index(drop=True) # xóa cột `contain_url`

Processor.printAfterProcess(reviews)

Shape: (15885, 3)
label
1    11961
0     2156
2     1768
Name: count, dtype: int64


<hr>

* Tiếp theo, ta sẽ kiểm tra các comment mà chữ IN HOA chiếm quá 50% độ dài comment, các comment này khả năng cao cũng là quảng cáo, vì người bán họ muốn làm nổi bật bình luận này lên so với các bình luận còn lại.

In [14]:
reviews['contain_adv'] = reviews['raw_comment'].apply(lambda cmt: Processor.containAdvertisement(cmt))

Processor.printAfterProcess(reviews, 'contain_adv')

Shape: (15885, 4)
contain_adv
False    15875
True        10
Name: count, dtype: int64


In [ ]:
filtered_reviews = reviews[reviews['contain_adv'] == True][['raw_comment', 'contain_adv']]
filtered_reviews

,raw_comment,contain_adv
179,#NAME?,True
3281,"SUPERCALIFRAGILISTICEXPIALIDOCIOUS\nMỌI THỨ KHÁ OK. CHẤT LIỆU VẢI, KHẢ NĂNG CO DÃN. ẢNH MÈO CUTE, CHỈ CÓ CỔ ÁO HƠI CHẬT VÀ BÓ 1 TÍ NÊN 4 SAO NHE",True
3611,VẢI NÓNG GIẶT 2 LẦN ĐÃ SỜM.K BẰNG ÁO 30K SHOP KHÁC,True
3872,VẢI NÓNG GIẶT 2 LẦN ĐÃ SỜM.K BẰNG ÁO 30K SHOP KHÁC,True
5305,BTh,True
7149,#NAME?,True
10484,K,True
11192,F,True
12414,"GIAO SAI MÀU RỒI ĐẶT CỔ ÁO MÀU VÀNG MÀ GIAO NGUYÊN CÁI ÁO TRẮNG,THẤY TỘI SHIPPER ĐI GIAO NÊN NHẬN LUÔN KHÔNG ĐỔI",True
14515,H,True


In [15]:
reviews = reviews.drop(columns=['contain_adv']).reset_index(drop=True) # xóa cột `contain_adv`

Processor.printAfterProcess(reviews)

Shape: (15885, 3)
label
1    11961
0     2156
2     1768
Name: count, dtype: int64


<hr>

* Nhìn qua các comment, ta sẽ thấy có các comment chứa emoji như hình dưới đây:<br>
  ![](./images/05.png)

* Đây là "vốn quý" góp phần làm tăng sức mạnh cho model, nếu ta thực hiện bước loại bỏ các kí tự đặc biệt trước khi ta tách các emoji ra, thì ta đã vô tình xóa luôn các emoji này, vì các emoji thực chất được xây dựng dựa trên các kí tự đặc biệt.
* Như hình trên, rõ ràng ta thấy được emoji góp phần ta hiểu được một comment là positive hay negative.
* Ta sẽ sử dụng một gói của python là `emojis`:
  ```shell
  pip3 install emojis
  ```
  gói này sẽ giúp ta tách các emoji ra khỏi bình luận.
* Ta sẽ chứa toàn bộ emoji của một comment qua cột tương ứng là `emoji`

In [16]:
reviews['emoji'] = reviews['raw_comment'].apply(lambda cmt: Processor.extractEmoji(cmt))

reviews['emoji']

0                                                                💞
1                                                                 
2                                                                 
3                                                                 
4                                                                 
5                                                                 
6                                                                 
7                                                                 
8                                                                 
9                                                             🤣 ❤️
10                                                                
11                                                                
12                                                                
13                                                                
14                                                            

In [17]:
",".join(reviews['emoji'])[:200]

'💞,,,,,,,,,🤣 ❤️,,,,,👌 😍,🥰,,,,,,😍,,🤣,🥰,❤️,❤️,🥰,,,❤️,,,,💗,,🥰,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,👍,,,,😢,,,,,,,🥲,,,,,,,,😔 🤧,,😞,,,,,,,,,,,,,,,,,,,,,,😒,,,,⭐ 🙂,,,,,,,,,,,,👍,,,,,,,,😊,,⭐,,,,,,,,,,,,'

<hr>

* Tiếp theo, ta sẽ loại bỏ dấu câu, kí tự đặc biệt

In [18]:
reviews['normalized_comment'] = reviews['normalized_comment'].apply(lambda cmt: Processor.removeSpecialLetters(cmt.lower()))

<hr>

* Tiếp theo, ta cần chuẩn lại các từ bị dupplicate như: _chờiiiiiii ơiiiiiii, xinhhhhhhh quá, đẹp xỉuuuuuuuuuuuu_ thành _chời ơi, xinh quá, đẹp xỉu_.
* Tuy nhiên có một vấn đề xảy ra, giả sử trong comment có các từ tiếng anh như "_feedback_", thì nó sẽ thành "_fedback_", nên ta sẽ thực hiện bước này ở phần sau:

```python
reviews['normalize_comment'] = reviews['normalize_comment'].apply(lambda cmt: Processor.removeDuplicateLetters(cmt))

```

<hr>

* Tiếp theo, chúng ta sẽ chuẩn lại một vài từ viết tắt cơ bản.
* File `modules/dependencies/abbreviate.txt` chứa các từ viết tắt cơ bản mà giới trẻ hay dùng comment, ta có thể bổ sung theo thời gian.

In [19]:
# xây dựng dictionary cho các từ viết tắt
abbreviate = Utils.buildDictionaryFromFile("./modules/dependencies/abbreviate.txt")

# test
abbreviate['okela']

'ok'

In [20]:
reviews['normalized_comment'] = reviews['normalized_comment'].apply(lambda cmt: Processor.replaceWithDictionary(cmt, abbreviate))

<hr>

* Bây giờ ta sẽ tiến hành xóa các từ vô nghĩa trong comment, ví dụ như hình dưới đây:<br>
  ![](./images/06.png)
  những bình luận này mặc dù thuộc lớp positive nhưng nó là các noise sample, có thể các comment này dùng để comment cho có để nhận shopee xu khi đánh giá sản phẩm.
* Kế tiếp, ta nên xóa các sample mà khả năng cao ko là tiếng việt, vì sao ta làm bước này, đơn giản thôi đây là shopee việt nam, và các comment cố ý bằng tiếng anh, tiếng hàn, tiếng trung sẽ là các noise sample khiến model ta bị giảm performance.
* Có thể sử dụng các package như `textblob`, `googletrans`,... các package này chứa các function giúpz
detect language cho text, tuy nhiên hạn chế là chúng chỉ cho tối đa khoảng 200 request một ngày thôi, và số mẫu của chúng ta hiện tại là quá lớn. Ở đây ta có file `modules/dependencies/vocabulary.txt` chứa hơn 17000 từ đơn phổ biến của tiếng việt.
* Vậy cách đơn giản hơn là ta có thể xây dựng một dictionary chứa các từ đơn của tiếng việt, với mỗi comment, nếu số lượng từ ko tìm thấy trong dictionary này lớn hơn số từ được tìm thấy trong dictionary thì khả năng cao đây là một comment làm màu.
* Tuy nhiên, vẫn có một vài từ tiếng anh mà ta cần giữ lại như shipper, ta sẽ sử dụng package `enchant` để check một từ có phải là từ tiếng anh hay không.
  ```shell
  pip3 install pyenchant
  ```
* Ở các bước phía trên, ta đã đề cập đến việc xóa các từ bị dupplicate kí tự, ta sẽ thực hiện nó ở trong bước này.

In [21]:
# hơn 17 ngàn từ đơn trong tiêng việt
vocabularies = Utils.buildDictionaryFromFile('./modules/dependencies/vocabulary.txt', True)
english_voca = enchant.Dict('en_US') # english if a word is english

In [22]:
reviews['normalized_comment'] = reviews['normalized_comment'].apply(lambda cmt: Processor.removeNoiseWord(cmt, vocabularies, english_voca))

<hr>

* Train test split, ta thấy rằng giữa hai nhóm positive và negative có chênh lệnh lớn, nên tập train data của ta sẽ bằng $0.8 * \min(\mathrm{size}(positive), \mathrm{size}(negative)) * 2$.

In [ ]:
half_min_size = min(reviews['label'].value_counts())

half_min_size

7247

In [ ]:
reviews_positive = reviews[reviews['label'] == 1]
reviews_negative = reviews[reviews['label'] == 0]

reviews_positive = shuffle(reviews_positive)
reviews_positive = reviews_positive.reset_index(drop=True)

In [ ]:
positive_index = random.sample(range(0, reviews_positive.shape[0]), half_min_size)

positive_index[:10]

[2650, 1308, 6365, 6472, 3110, 4583, 193, 3680, 1969, 6649]

In [ ]:
reviews_positive2 = reviews_positive.iloc[positive_index,:]

reviews_positive2.head()

,raw_comment,rating,label,normalized_comment,contain_url,contain_adv,emoji
2650,Mình cao m58 nặng 50kh bth mặc đồ size ...,5.0,1,cao nặng không bình thường mặc đồ s...,0,False,
1308,Đúng với mô tả: yepppMàu sắc: trắngS...,4.0,1,đúng mô tả sắc phẩm rất đẹp luôn hi...,0,False,🥹
6365,Quần đẹp nhưng hơi nóng :(((,4.0,1,quần đẹp nhưng hơi nóng,0,False,
6472,"Ok hàng giao đúng, bên trong có một cái...",5.0,1,ok hàng giao đúng bên trong có một cái ...,0,False,
3110,Chất liệu: hợp lýMàu sắc: trắngĐú...,5.0,1,chất liệu hợp sắc mô tả liệu mát,0,False,


* Đây là tập data mà hai nhóm positive và negative cân bằng nhau

In [ ]:
normalize_reviews = pd.concat([reviews_negative, reviews_positive2], axis=0)
normalize_reviews = normalize_reviews.reset_index(drop=True)

Processor.printAfterProcess(normalize_reviews)
normalize_reviews.head()

Shape: (12176, 7)
label
0    6088
1    6088
Name: count, dtype: int64


,raw_comment,rating,label,normalized_comment,contain_url,contain_adv,emoji
0,nan,NaN,0,nan,0,False,
1,Shop giao nhầm size. Áo from khá rộng nh...,3.0,0,giao nhầm size áo from khá rộng nhưng k...,0,False,
2,Áo form ổn. Khá xinh. Nhưng mà giao hàn...,3.0,0,áo form ổn khá xinh nhưng mà giao hàng ...,0,False,
3,Áo đẹp\nNhưng tay áo 1 bên trắng 1 bên...,3.0,0,áo đẹp nhưng tay áo bên trắng bên vàn...,0,False,
4,"Áo 2 lớp nhưng mỏng, có nhiều chỉ thu...",3.0,0,áo lớp nhưng mỏng có nhiều chỉ thừa...,0,False,


* Ghi ra file

In [ ]:
normalize_reviews.to_csv("./data/normalize_reviews.csv", index=False,encoding="utf-8-sig")

* Bây giờ ta sẽ ghi phần bù còn lại của `review_positive` vào file, ta có thể dùng nó cho việc evaluate model sau này.

In [ ]:
reviews_positive3 = reviews_positive[~reviews_positive.index.isin(positive_index)]
reviews_positive3 = reviews_positive3.reset_index(drop=True)

reviews_positive3.head()

,raw_comment,rating,label,normalized_comment,contain_url,contain_adv,emoji
0,Đúng với mô tả: giống 80%Màu sắc: đe...,4.0,1,đúng mô tả giống màu sắc đẹp mặc ra...,0,False,
1,Đúng với mô tả: chuẩn với mô tảMàu...,5.0,1,đúng mô tả chuẩn mô sắc màu liệu ba...,0,False,😘
2,Giao hàng nhanh\nSợ đặt rộng quá nhưn...,5.0,1,giao hàng nhanh sợ đặt rộng quá nhưng...,0,False,
3,Chất liệu: thua k biết là vải gì như...,4.0,1,chất liệu thua không biết là vải gì ...,0,False,
4,Hàng đẹp dày dặn. Mặc thích. Phong cá...,5.0,1,hàng đẹp dày dặn mặc thích phong cách...,0,False,


In [ ]:
reviews_positive3.to_csv("./data/complement_positive_reviews.csv", index=False, encoding="utf-8-sig")